In [99]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import os
from openai import OpenAI

In [100]:
with open(r"C:\Users\maxim.oweyssi\Energy Saving Trust\Lutz Lemmer - Sample_AI_Chatbot_Documents\apikey.txt","r") as r:
    lutzkey = r.readlines()
os.environ['OPENAI_API_KEY'] = lutzkey[0]

## Load embedding from CSV

In [101]:
df = pd.read_csv('embedding.csv')
df['ada_embedding'] = df.ada_embedding.apply(eval).apply(np.array)

In [102]:
client = OpenAI()

def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model,dimensions=512).data[0].embedding

def search_term_cosine_similarity(query, documents, vectorizer):
    query_vector = vectorizer.transform([query])
    document_vectors = vectorizer.transform(documents)
    similarities = cosine_similarity(query_vector, document_vectors).flatten()
    return similarities

## Create embedding matrix and embed the query vector

In [103]:
prompt = "How can I increase energy efficiency of my home?"
embedded_prompt = np.array(get_embedding(prompt)).reshape(1, -1)
embedding_matrix = np.array(df['ada_embedding'].to_numpy().tolist())

## Find the closest match within the embedded data

In [105]:
similarities = cosine_similarity(embedded_prompt, embedding_matrix).flatten()

def find_closest_matches(similarities, paragraphs,page_numbers, document_names):
    # Calculate cosine similarity

    # Sort by similarity in descending order
    sorting_indices = sorted(range(len(similarities)), key=lambda k: similarities[k], reverse=True)
    sorted_page_numbers = [page_numbers[i] for i in sorting_indices]
    sorted_document_names = [document_names[i] for i in sorting_indices]
    sorted_paragraphs = [paragraphs[i] for i in sorting_indices]
    sorted_similarities = [similarities[i] for i in sorting_indices]
    return sorted_paragraphs, sorted_page_numbers, sorted_document_names, sorting_indices, sorted_similarities

sorted_paragraphs, sorted_page_numbers, sorted_document_names, sorting_indices, sorted_similarities = find_closest_matches(similarities,
                                                                                                                           df['Text'],
                                                                                                                           df['Page'],
                                                                                                                           df['Document']
                                                                                                                           )


## Query chat GPT to answer the question based on information from files

In [106]:
def GPTprompt(pageno):
    text1 = "Please could you answer my question based on the text from '" + sorted_document_names[pageno][0:-4] + "' on " + sorted_page_numbers[pageno] + "?\n"
    text2 = "Answer my questions based on the text, but phrase your answers as your own and refer to the document as if I haven't provided it to you.\n"
    text3 = "If this text does not have enough information to answer in a full and unbiased way it is bad. This would happen for example if it comes from a page with not enough text where many images could have been. \nIn this case, only reply 'ERROR999' to me and nothing else. I will give you another page and we try again.\n"
    text4 = "\nMy question is:\n\n" + prompt + "\n\n"
    text5 = "Text is here:\n\n" + sorted_paragraphs[pageno]
    return(text1+text2+text3+text4+text5)

In [107]:

while True:
    pageno = 0
    response = client.chat.completions.create(
            model="gpt-3.5-turbo-0125",
            messages=[
                {"role": "system", "content": "You are a helpful assistant, working for the Welsh government providing people with information on saving energy."},
                {"role": "user", "content": GPTprompt(pageno)}
            ]
        )
    response_text = response.choices[0].message.content
    if response_text=='ERROR999':
        # Increment page number and update the prompt
        print("ERROR")
        print(response_text)
        pageno+=1
    else:
        # If no ERROR999 encountered, return the response
        print(pageno)
        print(response_text)
        break

0
To increase the energy efficiency of your home, it is important to focus on implementing energy efficiency measures such as improving insulation, reducing draughts, and upgrading heating systems. These measures can lead to reduced carbon emissions, lower energy bills, and a more comfortable living environment. While some homes may have basic measures like cavity wall and loft insulation, there is still a need for more comprehensive solutions for properties that are considered 'hard to treat.' This includes implementing measures like solid wall insulation, floor insulation, and secondary glazing. New technical approaches, materials, and funding mechanisms are necessary to address the energy efficiency challenges faced by older buildings, especially those with traditional construction techniques. By investing in energy-saving upgrades and adopting innovative solutions like those offered by Q-Bots, significant energy savings can be achieved in older buildings.
